# Scale Variations Investigations

## Some theory first

Let's recap: to interpolate we need only need the list of x-points $\mathbb G = \{x_j\}$ if we assume logarithmic interpolation by default, and, by default polynomial degree 4. Especially the interpolation polynomials $p_j(x)$ are uniquely defined by this. The full spell-out on the procedure is in the eko docs. Instead, let's turn our attention to the actual interpolation procedure.

First, I will introduce a more formal notation (that maybe we should inherit in the docs). The notation will consist in putting consistent indices whose order and position (up or down) matters.
- given a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$, I define $f_j = f(x_j)$, i.e. the explicit evaluation of the function $f(x)$ at an interpolation point $x_j$. As for the grid I use the notation $\{f_j\}$ to refer to all points as a set.
- given a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$, I define a new function $f^j : [0,1] \to \mathbb R : x\to f^j(x) = (f \otimes p_j)(x)$, i.e. the multiplicative convolution of $f$ with the interpolation polynomials $p_j$.
- The interpolation polynomials $p$ themselves are an exception to those definitions and I will continue to use them with a lower index, but refer to the ordinary function as defined above.
- I define the "identity function" $e : [0,1] \to \mathbb R : x\to e(x) = \delta(1-x)$, and we find $e^j(x) = p_j(x)$. Moreover, we have ${e^j}_k = (e^j)_k = \delta_{jk}$ with $\delta_{jk}$ the usual Kronecker delta due to the properties of the interpolation construction.

Now, we can write down the interpolation of a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$:
$$ f(x) \sim \bar f(x) = f_j e^j(x) $$
using Einstein summming convention as usual (Note that as in general relativity, there is an exact match between an upper and a lower index). Note that $\bar f$ can appoximate $f$ only where there points in $\mathbb G$ and since $\mathbb G$ is finite (by construction) we loose some information in the small-x regime.

Next, we can turn to yadism: the DIS cross section $\sigma(x)$ is given as the multiplicative convolution between a PDF $f(x)$ and a coefficient function $c(z)$:
$$ \sigma(x) = (f \otimes c)(x) $$
Here and in the following we neglect any additional parameters the functions may cary, such as scales.

What we actually do is a interpolated version of the master formula $\sigma(x) = f_j c^j(x)$, i.e. we compute $c^j(x)$ the convolution of the (analytic) coefficient function $c$ with the interpolation polynomial $e^j$ with respect to the (hadronic) Bjorken-x.

How about scale variations, i.e. what if $c$ itself is a convolution? No problem! Given $d = P \otimes c$ we find
$$d^k(x) = (P \otimes c \otimes e^k)(x) = (P^k \otimes c)(x) = {P^k}_j c^j(x) $$
where we approximated the well-behaved function $P^k$ with a full set of polynomials. So our strategy of computing all $c^j(x)$ first and then supply them with a convolution matrix for the splitting kernels is good.

Finally, we can turn to the Mellin transformation: given a sufficiently well-behaved function $f : [0,1] \to \mathbb R : x\to f(x)$, I define
$$ \tilde f(N) = \mathcal M[f(x)](N) = \int\limits_0^1 x^{N-1}f(x)\,dx $$
Note that the Mellin transformation is running over the full domain. The Mellin transformation turns multiplicative convolutions into ordinary products, so, e.g., for the DIS master formula, we find $\tilde \sigma(N) = \tilde f (N) \cdot \tilde c(N)$ (where, $\tilde\sigma$ contains information about all Bjorken-x). Specifically, using the interpolation language we find $\tilde g^j(N) = \tilde g(N) \tilde e^j(N)$.

## What happens in practice

let's prepare a yadism run - we use the simplest options:
- we compute only F2_light
- we use NLO
- we use FFNS5
- we only allow photon exchange

In [1]:
import copy
import yadism
import yadism.log
import yaml
import numpy as np
from eko import interpolation

t = {'ID': 208,
 'PTO': 1,
 'FNS': 'FFNS',
 'DAMP': 0,
 'IC': 1,
 'IB': 0,
 'ModEv': 'TRN',
 'ModSV': 'unvaried',
 'XIR': 1.0,
 'XIF': 2.0,
 'fact_to_ren_scale_ratio': 1.0,
 'NfFF': 5,
 'MaxNfAs': 5,
 'MaxNfPdf': 5,
 'Q0': 1.65,
 'alphas': 0.118,
 'Qref': 91.2,
 'nf0': None,
 'nfref': None,
 'QED': 0,
 'alphaqed': 0.007496252,
 'Qedref': 1.777,
 'SxRes': 0,
 'SxOrd': 'LL',
 'HQ': 'POLE',
 'mc': 1.51,
 'Qmc': 1.51,
 'kcThr': 0.0,
 'mb': 4.92,
 'Qmb': 4.92,
 'kbThr': 0.0,
 'mt': 172.5,
 'Qmt': 172.5,
 'ktThr': 1.0,
 'CKM': '0.97428 0.22530 0.003470 0.22520 0.97345 0.041000 0.00862 0.04030 0.999152',
 'MZ': 91.1876,
 'MW': 80.398,
 'GF': 1.1663787e-05,
 'SIN2TW': 0.23126,
 'TMC': 0,
 'MP': 0.938,
 'Comments': 'NNPDF4.0 NLO alphas=0.118',
 'global_nx': 0,
 'EScaleVar': 1,
 'kDIScThr': 0.0001,
 'kDISbThr': 0.0001,
 'kDIStThr': 1.0}

o = {'PolarizationDIS': 0.0,
 'ProjectileDIS': 'electron',
 'PropagatorCorrection': 0.0,
 'TargetDIS': 'proton',
 'interpolation_is_log': True,
 'interpolation_polynomial_degree': 4,
 'interpolation_xgrid': [],
 'observables': {'F2_light': []},
 'prDIS': 'EM'}

# setup grids
grids = [
    interpolation.make_lambert_grid(30),
    interpolation.make_lambert_grid(60),
    interpolation.make_lambert_grid(90),
    interpolation.make_lambert_grid(120),
    interpolation.make_lambert_grid(150),
]
# run yadism
outs = []
yadism.log.setup(log_to_stdout=False)
for grid in grids:
    oo = copy.deepcopy(o)
    oo["interpolation_xgrid"] = grid
    obs = []
    for x in grid:
        obs.append(dict(Q2=300,x=x))
    oo["observables"]["F2_light"] = obs
    outs.append(yadism.run_yadism(t,oo))

┌────────────────────────────────────┐                                       
                                      │                                    │                                       
                                      │ __     __       _ _                │                                       
                                      │ \ \   / /      | (_)               │                                       
                                      │  \ \_/ /_ _  __| |_ ___ _ __ ___   │                                       
                                      │   \   / _` |/ _` | / __| '_ ` _ \  │                                       
                                      │    | | (_| | (_| | \__ \ | | | | | │                                       
                                      │    |_|\__,_|\__,_|_|___/_| |_| |_| │                                       
                                      │                                    │                                       
                                      └────────────────────────────────────┘

Plan

 • F2_light at 30 pts

Calculation

yadism took off! please stay tuned ...

...

Calculation

yadism took off! please stay tuned ...

[10:27:34] INFO     computing P_qq_0 - took: 3.362045 s                                      ]8;id=48869;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=964042;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py#80\80]8;;\

[10:27:36] INFO     computing P_qg_0 - took: 2.457000 s                                      ]8;id=314825;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=48439;file:///home/felix/Physik/N3PDF/DIS/yadism/src/yadism/esf/scale_variations.py#80\80]8;;\

took 39.87 s

In [2]:
# let's prepare the mellin transformation
from eko.interpolation import InterpolatorDispatcher
from eko.anomalous_dimensions import as1
from eko.harmonics import S1

def mel2(out, ys, n):
    interp = InterpolatorDispatcher.from_dict(out)
    xgrid = out["interpolation_xgrid"]
    lnxmin = np.log(xgrid[0])
    res = 0.
    for y, bf in zip(ys, interp):
        pj = bf(n, lnxmin) * np.exp(n * lnxmin) # remember we need to counteract the x^(-N)
        res += y * pj
    return res

In [3]:
# let's extract some elements from the operator!
# - j refers to the ESF, that is to Bjorken-x
# - the index for orders is the usual PineAPPL order 
# - the 0 thereafter refers to the operator (instead of the error)
# - next is the flavor index, where we sort antitop,...,antidown, gluon, down, ... top
#   so charm is -3 and gluon 7
# - finally I'm selecting a single basis function
def collect(order, pid, bf):
    data = []
    for out in outs:
        xgrid = out["interpolation_xgrid"]
        data.append([out["F2_light"][j].orders[order][0][pid][bf] for j in range(len(xgrid))])
    return data
locbf10s = collect((0,0,0,0),-3,10)
pqqcbf10s = collect((1,0,0,1),-3,10)
pqgbf10s = collect((1,0,0,1),7,10)
locbf20s = collect((0,0,0,0),-3,20)
pqqcbf20s = collect((1,0,0,1),-3,20)
pqgbf20s = collect((1,0,0,1),7,20)

In [4]:
# next, we can compute some Mellin trafos: I choosen (almost) at random the integers from N=1..10
ns = np.linspace(1,15,30)
# first the polynomials, which we can select with a sufficiently good "delta function"
mel2bf10 = np.array([[mel2(out,[0]*10 + [1] + [0]*39,n) for n in ns] for out in outs])
mel2bf20 = np.array([[mel2(out,[0]*20 + [1] + [0]*29,n) for n in ns] for out in outs])
# then all the elements
mel2lobf10 = np.array([[mel2(out, locbf10,n) for n in ns] for out, locbf10 in zip(outs, locbf10s)])
mel2pqqcbf10 = np.array([[mel2(out, pqqcbf10,n) for n in ns] for out, pqqcbf10 in zip(outs, pqqcbf10s)])
mel2pgqgbf10 = np.array([[mel2(out, pqgbf10,n) for n in ns] for out, pqgbf10 in zip(outs, pqgbf10s)])
mel2lobf20 = np.array([[mel2(out, locbf20,n) for n in ns] for out, locbf20 in zip(outs, locbf20s)])
mel2pqqcbf20 = np.array([[mel2(out, pqqcbf20,n) for n in ns] for out, pqqcbf20 in zip(outs, pqqcbf20s)])
mel2pqgbf20 = np.array([[mel2(out, pqgbf20,n) for n in ns] for out, pqgbf20 in zip(outs, pqgbf20s)])
# finally, some anomalous dimensions
gns = np.array([as1.gamma_ns(n, S1(n)) for n in ns])
gqg = np.array([as1.gamma_qg(n, 5) for n in ns])

[10:28:10] INFO     Interpolation: number of points = 30, polynomial degree = 4, logarithmic = ]8;id=396189;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=360147;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    True                                                                                           

           INFO     Interpolation: number of points = 30, polynomial degree = 4, logarithmic = ]8;id=888167;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=145414;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    True                                                                                           

           INFO     Interpolation: number of points = 30, polynomial degree = 4, logarithmic = ]8;id=309458;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=539382;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    True                                                                                           

           INFO     Interpolation: number of points = 30, polynomial degree = 4, logarithmic = ]8;id=407394;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=235074;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    True                                                                                           

[10:28:11] INFO     Interpolation: number of points = 30, polynomial degree = 4, logarithmic = ]8;id=944854;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=628673;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    True                                                                                           

...

           INFO     Interpolation: number of points = 150, polynomial degree = 4, logarithmic  ]8;id=10817;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=204969;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    = True                                                                                         

           INFO     Interpolation: number of points = 150, polynomial degree = 4, logarithmic  ]8;id=979074;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=932709;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    = True                                                                                         

           INFO     Interpolation: number of points = 150, polynomial degree = 4, logarithmic  ]8;id=618010;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=236732;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    = True                                                                                         

           INFO     Interpolation: number of points = 150, polynomial degree = 4, logarithmic  ]8;id=817516;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=408969;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    = True                                                                                         

           INFO     Interpolation: number of points = 150, polynomial degree = 4, logarithmic  ]8;id=418780;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=341099;file:///home/felix/.cache/pypoetry/virtualenvs/yadism-QljnF84o-py3.10/lib/python3.10/site-packages/eko/interpolation.py#469\469]8;;\
                    = True                                                                                         

### Leading order

Now, we can check leading order where we have $c(z) = e_q^2 e(z)$ and for charm (i.e. an uplike quark), we have $e_q^2 = 4/9$.

Let's recall we compute $c^j(x)$, so, e.g., we have $c^{10}(x) = {c^{10}}_j e^j(x)$ with `locbf10` = $\{{c^{10}}_j\}$, and so $\tilde c^{10}(N) = {c^{10}}_j \tilde e^j(N)$. Ultimatively we have $c^{10}(x) = 4/9{e^{10}}_j e^j(x) = 4/9 e^{10}(x)$ and so $\tilde c^{10}(N)/\tilde e^{10}(N) = 4/9$.

I'm already failing to find this! I have an additional division by $x_j$:

In [5]:
np.set_printoptions(linewidth=200)

In [10]:
print("p_10")
print((((mel2lobf10/mel2bf10).T/np.array([g[10] for g in grids])).T).T)
print("p_10")
print((((mel2lobf20/mel2bf20).T/np.array([g[20] for g in grids])).T).T)

p_10
[[0.44444444 0.44444444 0.44444444 0.44444444 0.44444445]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444445]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444445]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444445]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444445]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444446]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444447]
 [0.44444444 0.44444445 0.44444444 0.44444444 0.44444447]
 [0.44444

### Pqq and Pqg

Since LO is so simple we should also be able to predict the scale variation coming with $P_{qq}$ (and also with $P_{qg}$).
We have $c^j(x)=e_q^2 {P_{\text{ns}}}^j(x)$ and so $\tilde c^{10}(N) = 4/9 \tilde P_{\text{ns}}(N) \tilde e^{10}(N)$.

And here I have immediately another problem: I can not seperate the $4/9$ at $N=1$, because $P_{\text{ns}}(N=1)=0$!

In [8]:
print("p_10")
print((((mel2pqqcbf10/mel2bf10).T/np.array([g[10] for g in grids])).T/(-4/9*gns)).T)
print("p_20")
print((((mel2pqqcbf20/mel2bf20).T/np.array([g[20] for g in grids])).T/(-4/9*gns)).T)

p_10
[[-9.26618487e+14+0.j -8.57855718e+14+0.j -7.49299443e+14+0.j  7.36605498e+13+0.j  3.41026409e+15+0.j]
 [ 2.58312080e+00-0.j  2.26342513e+00-0.j  2.23778685e+00-0.j  2.12846565e+00-0.j  1.61575373e+00-0.j]
 [ 2.00053358e+00-0.j  1.60798294e+00-0.j  1.58835962e+00-0.j  1.57828069e+00-0.j  1.53703609e+00-0.j]
 [ 1.96166017e+00-0.j  1.40391684e+00-0.j  1.37662843e+00-0.j  1.37157444e+00-0.j  1.36491038e+00-0.j]
 [ 2.12757530e+00-0.j  1.31434837e+00-0.j  1.27523382e+00-0.j  1.26895472e+00-0.j  1.26635997e+00-0.j]
 [ 2.47873337e+00-0.j  1.27321732e+00-0.j  1.21805748e+00-0.j  1.20924729e+00-0.j  1.20676675e+00-0.j]
 [ 2.58170953e+00-0.j  1.25942629e+00-0.j  1.18332761e+00-0.j  1.17108193e+00-0.j  1.16785818e+00-0.j]
 [ 3.52060643e+00-0.j  1.26471904e+00-0.j  1.16193059e+00-0.j  1.14528862e+00-0.j  1.14093123e+00-0.j]
 [ 4.51902217e+00-0.j  1.28571811e+00-0.j  1.14947511e+00-0.j  1.12734799e+00-0.j  1.12152756e+00-0.j]
 [ 5.88833553e+00-0.j  1.32131240e+00-0.j  1.14365944e+00-0.j  1.114

In [9]:
print("p_10")
print((((mel2pgqgbf10/mel2bf10).T/np.array([g[10] for g in grids])).T/(-4/9*gqg/2)).T)
print("p_20")
print((((mel2pqgbf20/mel2bf20).T/np.array([g[20] for g in grids])).T/(-4/9*gqg/2)).T)

p_10
[[ 5.33980094e-01  5.47694750e-01  5.55078285e-01  8.03536673e-01  1.64946682e+00]
 [ 6.27375610e-01  6.79701872e-01  6.82093748e-01  7.06533589e-01  8.03368196e-01]
 [ 6.24838295e-01  7.59919408e-01  7.64587405e-01  7.68759766e-01  7.85728518e-01]
 [ 5.12601471e-01  8.09113522e-01  8.19076439e-01  8.21024696e-01  8.25025701e-01]
 [ 2.48716064e-01  8.37181763e-01  8.56366659e-01  8.58986563e-01  8.60496703e-01]
 [-2.58679337e-01  8.48419806e-01  8.82401572e-01  8.86837486e-01  8.88099778e-01]
 [-6.59370077e-01  8.43901259e-01  9.00388037e-01  9.07727861e-01  9.09464403e-01]
 [-2.17184334e+00  8.22640703e-01  9.12010507e-01  9.23606805e-01  9.26255052e-01]
 [-4.17688464e+00  7.82121634e-01  9.18063230e-01  9.35647462e-01  9.39640798e-01]
 [-7.24757848e+00  7.18490157e-01  9.18780135e-01  9.44544544e-01  9.50394333e-01]
 [-1.19287867e+01  6.26559073e-01  9.14009760e-01  9.50689411e-01  9.59019598e-01]
 [-1.90150305e+01  4.99687706e-01  9.03308425e-01  9.54271418e-01  9.65841374e-01]